<a href="https://colab.research.google.com/github/OWEN-JUN/keras_/blob/master/day0810_colab_cifar10_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import *
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, train_test_split


import numpy as np
import os
import tensorflow as tf


data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.08,
    height_shift_range=0.08,
    horizontal_flip=True
)



(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()


# X_train, _, Y_train, _ = train_test_split(X_train, Y_train, test_size=0.994)
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

X_train = X_train.reshape(X_train.shape[0], 32 ,32, 3)  
X_test = X_test.reshape(X_test.shape[0], 32 ,32, 3)


Y_train = np_utils.to_categorical(Y_train)      
Y_test = np_utils.to_categorical(Y_test)        


IMG_CHANNERLS = 3
IMG_ROWS = 32
IMG_COLS = 32




# print(X_train[0])
# data_scaling(X_train)
# print(X_train[0])
# X_train = X_train.reshape(X_train.shape[0], 32 ,32 ,3).astype('float32') / 255
# print(X_train[0])

def build_network_cnn(keep_prob=0.1, optimizer='adam', conv1 = 20, conv2=30):
    
    inputs = Input(shape=(32,32,3), name='input')
    
    x = Conv2D(conv1,(3,3),padding="same",activation="relu", input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNERLS))(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(conv1, (3,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(keep_prob)(x)
    
    x = Conv2D(conv2,(3,3),padding="same",activation="relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(conv2, (3,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(keep_prob)(x)
    
    
#     x = Conv2D(conv3,(3,3),padding="same",activation="relu")(x)
#     x = BatchNormalization()(x)
#     x = Conv2D(conv3, (3,3), activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling2D(pool_size=2)(x)
#     x = Dropout(keep_prob)(x)

    
    
    
    
    
    x = Flatten()(x)
    prediction = Dense(10, activation='softmax', name='output')(x)
    
    model = Model(inputs=inputs, outputs=prediction)
 

    model.compile(loss='categorical_crossentropy',  
                optimizer=optimizer,
                metrics=['accuracy'])
#     model.fit_generator(data_generator.flow(X_train, Y_train, batch_size=256),
#                         steps_per_epoch=100,     
#                         epochs=3,
#                         verbose=1)  
    
    return model


def create_hyperparameters():
    batches = [10,20,30,40,50]
    optimizers = ['rmsprop', 'adam', 'adadelta']
    dropout = np.linspace(0.05,0.5, 10)
    epochs = [10,50,100,150, 200]
    conv1 = [30,40,50,60,70]
    conv2 = [30,40,50,60,70]
#     conv3 = [30,40,50,60,70]
    return {"model__batch_size":batches, "model__optimizer":optimizers, "model__keep_prob":dropout, "model__epochs":epochs,"model__conv1":conv1,"model__conv2":conv2}      # Map 형태로 반환

# def data_scaling(x_data):
#     x_data = x_data.reshape(x_data.shape[0], 32* 32* 3) 
#     MinMaxScaler()
#     x_data = x_data.reshape(x_data.shape[0], 32, 32, 3)  


In [12]:
# score : {'model__optimizer': 'adadelta', 'model__keep_prob': 0.05, 'model__epochs': 200, 'model__conv2': 60, 'model__conv1': 30, 'model__batch_size': 50}
early = EarlyStopping(monitor="loss", patience=10, mode="auto")
model = build_network_cnn(keep_prob=0.05, optimizer='adadelta', conv1 = 30, conv2=60)
model.fit(X_train,Y_train,batch_size=50, epochs=200, callbacks=[early])

print("acc:", model.evaluate(X_test,Y_test))


Epoch 1/200
50000/50000 [==============================] - 11s 216us/step - loss: 1.3421 - acc: 0.5447
Epoch 2/200
50000/50000 [==============================] - 10s 191us/step - loss: 0.8711 - acc: 0.6955
Epoch 3/200
50000/50000 [==============================] - 10s 191us/step - loss: 0.7305 - acc: 0.7472
Epoch 4/200
50000/50000 [==============================] - 10s 191us/step - loss: 0.6389 - acc: 0.7799
Epoch 5/200
50000/50000 [==============================] - 10s 192us/step - loss: 0.5730 - acc: 0.8013
Epoch 6/200
50000/50000 [==============================] - 9s 189us/step - loss: 0.5199 - acc: 0.8179
Epoch 7/200
50000/50000 [==============================] - 9s 190us/step - loss: 0.4729 - acc: 0.8353
Epoch 8/200
50000/50000 [==============================] - 9s 189us/step - loss: 0.4347 - acc: 0.8482
Epoch 9/200
50000/50000 [==============================] - 9s 189us/step - loss: 0.3994 - acc: 0.8606
Epoch 10/200
50000/50000 [==============================] - 10s 190us/step - 

In [15]:
print("acc:", model.evaluate(X_test,Y_test)[1])

10000/10000 [==============================] - 2s 154us/step
acc: 0.7393


In [19]:
def build_network_cnn(keep_prob=0.1, optimizer='adam', conv1 = 20, conv2=30, conv3=40):
    
    inputs = Input(shape=(32,32,3), name='input')
    
    x = Conv2D(conv1,(3,3),padding="same",activation="relu", input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNERLS))(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(conv1, (3,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(keep_prob)(x)
    
    x = Conv2D(conv2,(3,3),padding="same",activation="relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(conv2, (3,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(keep_prob)(x)
    
    
    x = Conv2D(conv3,(3,3),padding="same",activation="relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(conv3, (3,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(keep_prob)(x)

    
    
    
    
    
    x = Flatten()(x)
    prediction = Dense(10, activation='softmax', name='output')(x)
    
    model = Model(inputs=inputs, outputs=prediction)
 

    model.compile(loss='categorical_crossentropy',  
                optimizer=optimizer,
                metrics=['accuracy'])
  
    
    return model




# score : {'model__optimizer': 'adadelta', 'model__keep_prob': 0.2, 'model__epochs': 50, 'model__conv3': 40, 'model__conv2': 70, 'model__conv1': 60, 'model__batch_size': 30}
early = EarlyStopping(monitor="loss", patience=10, mode="auto")
model2 = build_network_cnn(keep_prob=0.2, optimizer='adadelta', conv1 = 60, conv2=70,conv3=40)
model2.fit(X_train,Y_train,batch_size=50, epochs=200, callbacks=[early])

print("acc:", model2.evaluate(X_test,Y_test))

Epoch 1/200
50000/50000 [==============================] - 18s 363us/step - loss: 1.4950 - acc: 0.4725
Epoch 2/200
50000/50000 [==============================] - 15s 307us/step - loss: 0.9588 - acc: 0.6610
Epoch 3/200
50000/50000 [==============================] - 15s 306us/step - loss: 0.7831 - acc: 0.7233
Epoch 4/200
50000/50000 [==============================] - 15s 306us/step - loss: 0.6885 - acc: 0.7594
Epoch 5/200
50000/50000 [==============================] - 15s 307us/step - loss: 0.6264 - acc: 0.7809
Epoch 6/200
50000/50000 [==============================] - 15s 306us/step - loss: 0.5751 - acc: 0.7989
Epoch 7/200
50000/50000 [==============================] - 15s 306us/step - loss: 0.5400 - acc: 0.8123
Epoch 8/200
50000/50000 [==============================] - 15s 305us/step - loss: 0.5087 - acc: 0.8247
Epoch 9/200
50000/50000 [==============================] - 15s 305us/step - loss: 0.4807 - acc: 0.8338
Epoch 10/200
50000/50000 [==============================] - 15s 307us/ste

KeyboardInterrupt: ignored

In [21]:
print("acc:", model2.evaluate(X_test,Y_test)[1])

10000/10000 [==============================] - 1s 144us/step
acc: 0.85
